# In this notebook, I construct novel sentences using LSTM network


In [1]:
import numpy as np
import torch.nn
import torch.nn.functional as F
import torch

### loading the data:

In [2]:
with open('data/anna.txt') as file:
    text = file.read()
    
text[0:50]

'Chapter 1\n\n\nHappy families are all alike; every un'

### define encoding function to encode words to numbers:

In [14]:
def encode(text):
    chars = tuple(set(text))
    int2char = dict(enumerate(chars))
    char2int = {char: i for i, char in int2char.items()}
    encoded = np.array([char2int[char] for char in text])
    return encoded

In [38]:
encoded = encode(text)
encoded[:50]

array([19, 82,  0, 57, 12,  5, 20, 63, 79, 42, 42, 42, 67,  0, 57, 57, 47,
       63,  2,  0, 24, 60, 81, 60,  5,  4, 63,  0, 20,  5, 63,  0, 81, 81,
       63,  0, 81, 60, 55,  5, 74, 63,  5, 21,  5, 20, 47, 63, 27, 80])

### one-hot encoding of the data:

In [39]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    print("n_labels = {0},    first dim = {1}".format(n_labels, np.multiply(*arr.shape)))
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

### getting batches of the data:

In [44]:
def get_batches(data, n_seq, n_steps):
    batch_size = n_seq * n_steps
    n_batches = len(data) // batch_size
    data = data[:n_batches*batch_size]
    data = data.reshape((n_seq, -1))
    
    for i in range(0, data.shape[1], n_steps):
        x = data[:, i:i+n_steps]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], data[:, i+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], data[:, 0]
        yield x, y

In [46]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[19 82  0 57 12  5 20 63 79 42]
 [63  0 24 63 80 66 12 63 76 66]
 [21 60 80 75 42 42 11 28  5  4]
 [80 63 10 27 20 60 80 76 63 82]
 [63 60 12 63 60  4 39 63  4 60]
 [63 15 12 63 51  0  4 42 66 80]
 [82  5 80 63 26 66 24  5 63  2]
 [74 63  6 27 12 63 80 66 51 63]
 [12 63 60  4 80 62 12 75 63 68]
 [63  4  0 60 10 63 12 66 63 82]]

y
 [[82  0 57 12  5 20 63 79 42 42]
 [ 0 24 63 80 66 12 63 76 66 60]
 [60 80 75 42 42 11 28  5  4 39]
 [63 10 27 20 60 80 76 63 82 60]
 [60 12 63 60  4 39 63  4 60 20]
 [15 12 63 51  0  4 42 66 80 81]
 [ 5 80 63 26 66 24  5 63  2 66]
 [63  6 27 12 63 80 66 51 63  4]
 [63 60  4 80 62 12 75 63 68 82]
 [ 4  0 60 10 63 12 66 63 82  5]]
